In [1]:
import json
import argparse
from easydict import EasyDict
from importlib import import_module

import gc
import warnings
from tqdm import tqdm
import os
import torch
import numpy as np
import random
import time
from torch.utils.data import DataLoader
import wandb

from mapcalc import calculate_map
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

from sklearn.model_selection import StratifiedKFold

In [2]:
def get_args(config):
    args = EasyDict()
    with open(f'./config/{config}.json', 'r') as f:
        args.update(json.load(f))
    
    return args
        
def killmemory():
    gc.collect()
    torch.cuda.empty_cache()

def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

def create_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)
        
def collate_fn(batch):
    return tuple(zip(*batch))

class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

def set_valid_gt(valid_dataset):
    gt = []

    for idx in range(len(valid_dataset)):
        data = valid_dataset[idx][1]
        gt.append({'boxes': data['boxes'].tolist(), 'labels': data['labels'].tolist()})

    return gt

def validation(valid_dataloader, model, device, resize, val_annotation):
    model.eval()
    outputs = []
    for images, targets, image_ids in valid_dataloader:

        images = list(image.float().to(device) for image in images)
        output = model(images)

        for out in output:
            outputs.append({'boxes': out['boxes'].tolist(), 'labels': out['labels'].tolist(), 'scores': out['scores'].tolist()})
    
    results = []
    cocoGt = COCO(val_annotation)
    for i, output in enumerate(outputs):
        file_name = cocoGt.loadImgs(cocoGt.getImgIds(imgIds=i))[0]['file_name']
        for bbox, score, label in zip(output['boxes'], output['scores'], output['labels']):
            results.append(
                {
                    'image_id': int(i),
                    'category_id': label,
                    'bbox': [
                        bbox[0] / (resize/512),
                        bbox[1] / (resize/512),
                        (bbox[2] - bbox[0]) / (resize/512),
                        (bbox[3] - bbox[1]) / (resize/512)
                    ],
                    'score': score
                }
            )
    '''
    ref
    https://www.programcreek.com/python/?code=potterhsu%2Feasy-faster-rcnn.pytorch%2Feasy-faster-rcnn.pytorch-master%2Fdataset%2Fcoco2017.py#
    https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/cocoeval.py
    '''
    cocoDt = cocoGt.loadRes(results)
    cocoEval = COCOeval(cocoGt, cocoDt, "bbox")
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()
    model.train()
    
    return cocoEval.stats, outputs


def get_logdata(valid_outputs, valid_dataset, idx):
    output = valid_outputs[idx]
    valid_box_data = []
    for i in range(len(output['labels'])):
        box = output['boxes'][i]
        label = output['labels'][i]
        score = output['scores'][i]

        log_data = {'position': {'minX':box[0],'maxX':box[2],'minY':box[1],'maxY':box[3]},
                    'class_id': label,
                    'box_caption': f'{label2class[label]}({score:.4f})',
                    'domain': 'pixel',
                    'scores': {'pred_score':score}}
        valid_box_data.append(log_data)

    gt_img = valid_dataset[idx][0]
    gt_data = valid_dataset[idx][1]
    gt_box_data = []
    for i in range(len(gt_data['labels'])):
        box = gt_data['boxes'][i].tolist()
        label = gt_data['labels'][i].item()

        log_data = {'position': {'minX':box[0],'maxX':box[2],'minY':box[1],'maxY':box[3]},
                    'class_id': label,
                    'box_caption': f'{label2class[label]}',
                    'domain': 'pixel',
                    'scores':{'pred_score':1}}
        gt_box_data.append(log_data)


    image_log = {"predictions":{'box_data':valid_box_data, 'class_labels':label2class}, 
                 "ground_truth":{'box_data':gt_box_data, 'class_labels':label2class}}

    img = wandb.Image(gt_img, boxes = image_log)
    
    return img

def create_fold_json(args, json_name, fold_idx):
    test_json = {}
    with open(f'/opt/ml/input/data/test.json', 'r') as f:
        test_json.update(json.load(f))

    new_json = {'info':[], 'licenses':[], 'images':[], 'categories':[], 'annotations':[]}
    new_json['info'] = test_json['info']
    new_json['licenses'] = test_json['licenses']
    new_json['categories'] = test_json['categories']
    
    coco = COCO(args.annotation)
    
    ann_idx = 0
    for idx, i in enumerate(fold_idx):
        img_info = coco.loadImgs(int(i))[0]
        img_info['id'] = idx
        new_json['images'].append(img_info)
        for ann in coco.getAnnIds(imgIds=i):
            anns_info = coco.loadAnns(ann)[0]
            anns_info['id'] = ann_idx
            anns_info['image_id'] = idx
            new_json['annotations'].append(anns_info)
            ann_idx += 1
    
    save_path = f'/opt/ml/input/data/{json_name}.json'
    with open(save_path, 'w') as f:
        json.dump(new_json, f)

In [3]:
config = 'config11'

In [4]:
args = get_args(config)

seed_everything(args.seed)
warnings.filterwarnings(action='ignore')
create_dir('./saved_model')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

classes = ("UNKNOWN","General trash","Paper","Paper pack","Metal","Glass","Plastic","Styrofoam","Plastic bag", "Battery", "Clothing")

train_augmentation_module = getattr(import_module("augmentation"), args.augmentation)
train_augmentation = train_augmentation_module(augp=args.augp, resize=args.resize)


dataset_module = getattr(import_module("dataset"), args.dataset)
kfold_dataset = dataset_module(args.annotation, args.data_dir, train_augmentation)

anns_cnt = []
for i in range(len(kfold_dataset)):
    anns_cnt.append(len(kfold_dataset[i][1]['boxes']))

skf = StratifiedKFold(n_splits=5)
for k, (train_idx, valid_idx) in enumerate(skf.split(kfold_dataset, anns_cnt)):
    killmemory()
    
    create_fold_json(args, 'train_kfold', train_idx)
    create_fold_json(args, 'valid_kfold', valid_idx)
    
    train_augmentation_module = getattr(import_module("augmentation"), args.augmentation)
    valid_augmentation_module = getattr(import_module("augmentation"), 'ValidAugmentation')
    train_augmentation = train_augmentation_module(augp=args.augp, resize=args.resize)
    valid_augmentation = valid_augmentation_module(resize=args.resize) 
    
    train_annotation = '/opt/ml/input/data/train_kfold.json'
    valid_annotation = '/opt/ml/input/data/valid_kfold.json'
    dataset_module = getattr(import_module("dataset"), args.dataset)
    train_dataset = dataset_module(train_annotation, args.data_dir, train_augmentation)
    valid_dataset = dataset_module(valid_annotation, args.data_dir, valid_augmentation)
    
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=0,
        collate_fn=collate_fn
    )
    valid_dataloader = DataLoader(
        valid_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=0,
        collate_fn=collate_fn
    )

    model_module = getattr(import_module("model"), args.model)
    model = model_module(num_classes = 11, args=args)
    model.to(device)

    optimizer_module = getattr(import_module("torch.optim"), args.optimizer)
    optimizer = optimizer_module(params = model.parameters(), lr=args.learning_rate)
    
    best_mAP50 = 0.3
    loss_hist = Averager()

    for epoch in range(args.epoch):
        print(f'* Epoch {epoch+1}')
        start_time = time.time()
        loss_hist.reset()

        model.train()
        for step, (images, targets, image_ids) in enumerate(train_dataloader):

            images = list(image.float().to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)

            losses = sum(loss for loss in loss_dict.values())
            loss_value = losses.item()

            loss_hist.send(loss_value)

            # backward
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
                
            print(f'  Step [{step+1}/{len(train_dataloader)}], Loss : {loss_value:.4f}', end="\r")

        train_time = time.time()-start_time
        print(f'  training time : {train_time:.4f}, train loss : {loss_hist.value} \n')

        if (epoch+1) % args.val_every == 0:
            print('* Start validation...', end="\r")
            start_time = time.time()
            mAP, valid_outputs = validation(valid_dataloader, model, device, args.resize[0], valid_annotation)
            print(f"  validation time : {time.time()-start_time:.4f}")


        if best_mAP50 < mAP[1]:
            print("  Best model saved")
            torch.save(model.state_dict(), f'./saved_model/({config}){args.config_name}_fold{k}.pth')
            best_mAP50 = mAP[1]

        print()


* Epoch 1
  training time : 482.9418, train loss : 0.9481314918467106 

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.067
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.137
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.060
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.023
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.068
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.133
  validation time : 60.0116

* Epoch 2
  training time : 482.8248, train loss : 0.773477517921506 

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.089
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.170
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.085
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.025
 Average Precision  (AP) @[ 